# Objective


The goal of this project is to independently value a loan portfolio as of **31 December 2020**, in support of the client’s year-end audit. The dataset includes **monthly loan origination amounts** and **observed repayments** for vintages between **June 2019 and December 2020**. 

To estimate the portfolio value, we followed a structured approach:

- **Historical Analysis:** Calculated the actual repayment percentages for each vintage across observed months by dividing repayments by the respective origination amounts.
- **Repayment Curve Modeling:** Derived a 20-month expected repayment curve using the historical averages of repayment behavior across all vintages.
- **Cash Flow Forecasting:** Applied the expected repayment percentages to each vintage’s origination amount to project future monthly cash flows from January 2021 onward.
- **Discounted Cash Flow Valuation:** Used a **5% annual discount rate** (converted to monthly) to discount the forecasted repayments back to their present value as of **31 December 2020**.
- **Variance Analysis:** Compared our model's valuation to the client-provided estimate of **CHF 84,993,122.67**, calculating both absolute and relative differences.
- **Audit Assessment:** Evaluated whether the difference exceeded the audit materiality threshold of **CHF 500,000**.

Our model estimated a portfolio value of **CHF 137.88 million**, exceeding the client's estimate by **CHF 52.88 million** (a **62.22%** relative difference). The discrepancy suggests differences in repayment assumptions, discounting horizon, or default expectations, which may require further reconciliation with the client for audit purposes.


## Assumptions: 
Customers keep repaying at the same rate as before.

There’s no increase in defaults or economic shocks.

No write-offs or losses are anticipated.

A 5% annual discount rate is used, converted to a monthly rate for DCF purposes.

The discount rate remains constant across the entire forecast period and does not reflect changing market or credit risk conditions.

## Data Cleaning

In [53]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from sklearn.compose import ColumnTransformer

In [54]:
df=pd.read_csv('Dataset.csv')

In [55]:
df

,Unnamed: 0,Origination Amount,31.05.2019,30.06.2019,31.07.2019,31.08.2019,30.09.2019,31.10.2019,30.11.2019,31.12.2019,...,31.03.2020,30.04.2020,31.05.2020,30.06.2020,31.07.2020,31.08.2020,30.09.2020,31.10.2020,30.11.2020,31.12.2020
0,31.05.2019,10018746.17,1443069.08,3332200.33,1328138.75,928085.74,736418.27,539403.31,427557.86,324459.32,...,116684.68,92699.67,63399.66,53265.12,37121.13,29787.10,24524.90,18085.94,16581.01,11442.97
1,30.06.2019,10868379.04,0.00,1392751.60,3011884.91,1237868.70,970929.28,892351.83,668767.02,505612.59,...,255222.42,198833.96,161996.73,138461.91,92346.68,79641.30,63457.44,52373.85,43374.70,37404.87
2,31.07.2019,10733932.61,0.00,0.00,1537650.24,2953335.55,1208316.08,879375.19,711016.84,658251.40,...,302575.54,258652.52,191798.05,170027.54,127574.33,110301.21,89766.69,64746.84,61408.92,50312.70
3,31.08.2019,12558727.02,0.00,0.00,0.00,1617681.94,4082016.00,1387474.94,1247623.59,886293.35,...,417223.56,336686.08,253556.20,200066.59,151859.74,109973.00,90228.14,70661.50,53102.83,47069.84
4,30.09.2019,14505071.44,0.00,0.00,0.00,0.00,1992242.84,3930445.60,1394620.78,1227905.58,...,628429.48,589692.85,457299.31,323764.87,288152.28,239872.99,192246.98,171550.69,142575.97,116853.05
5,31.10.2019,15652952.20,0.00,0.00,0.00,0.00,0.00,2289453.76,4682354.31,1659503.89,...,763523.36,742787.97,558085.95,461806.22,358671.23,281881.11,241719.91,182730.05,144953.58,119260.10
6,30.11.2019,15107713.30,0.00,0.00,0.00,0.00,0.00,0.00,2162283.09,4637701.69,...,930720.35,697500.94,667277.73,547749.09,387987.02,309448.86,283876.04,215635.85,185516.45,141560.57
7,31.12.2019,17004745.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2402403.37,...,1245452.39,1116505.94,803590.21,724956.28,545397.33,458832.95,393971.01,333818.53,286831.44,216447.57
8,31.01.2020,16794379.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1505493.21,1182983.95,955821.48,846061.73,683116.81,560572.44,468203.76,349067.92,309854.99,267813.78
9,29.02.2020,19217205.82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,6142911.08,1833677.81,1317065.75,1108494.37,918465.33,719913.69,587381.96,498801.31,371563.77,294941.22


## Calculate Repayment Percentages

In [56]:
# Calculate repayment percentages
rep_percentages_df = df.copy()
#calculate the repayment percentages from each column except the Unnamed and Origination
for col in rep_percentages_df.columns[2:]:  # Skip 'Unnamed' and 'Origination Amount' columns
    rep_percentages_df[col] = (df[col] / df['Origination Amount']) * 100

# Inspect the calculated repayment percentages
print(rep_percentages_df.head())

   Unnamed: 0  Origination Amount  31.05.2019  30.06.2019  31.07.2019  \
0  31.05.2019         10018746.17   14.403689   33.259654   13.256537   
1  30.06.2019         10868379.04    0.000000   12.814713   27.712365   
2  31.07.2019         10733932.61    0.000000    0.000000   14.325134   
3  31.08.2019         12558727.02    0.000000    0.000000    0.000000   
4  30.09.2019         14505071.44    0.000000    0.000000    0.000000   

   31.08.2019  30.09.2019  31.10.2019  30.11.2019  31.12.2019  ...  \
0    9.263492    7.350404    5.383940    4.267579    3.238522  ...   
1   11.389635    8.933524    8.210533    6.153328    4.652144  ...   
2   27.514012   11.256975    8.192479    6.624011    6.132435  ...   
3   12.880939   32.503422   11.047895    9.934316    7.057191  ...   
4    0.000000   13.734802   27.097044    9.614712    8.465354  ...   

   31.03.2020  30.04.2020  31.05.2020  30.06.2020  31.07.2020  31.08.2020  \
0    1.164664    0.925262    0.632810    0.531655    0.370517  

## Calculate the average expected repayment percentages

In [57]:
weighted_avg = (df[col] * df['Origination Amount']).sum() / df['Origination Amount'].sum()

# Calculate average (mean) repayment percentage per column (i.e., per period)
expected_rep_percentages = rep_percentages_df.iloc[:, 2:].mean().to_dict()

# Display the expected repayment percentages
print("Expected Repayment Percentages:")
for period, percentage in expected_rep_percentages.items():
    print(f"{period}: {percentage:.2f}%")

Expected Repayment Percentages:
31.05.2019: 0.72%
30.06.2019: 2.30%
31.07.2019: 2.76%
31.08.2019: 3.05%
30.09.2019: 3.69%
31.10.2019: 3.73%
30.11.2019: 4.04%
31.12.2019: 4.25%
31.01.2020: 4.24%
29.02.2020: 4.21%
31.03.2020: 4.32%
30.04.2020: 4.41%
31.05.2020: 4.33%
30.06.2020: 4.63%
31.07.2020: 4.55%
31.08.2020: 4.27%
30.09.2020: 4.45%
31.10.2020: 3.99%
30.11.2020: 4.21%
31.12.2020: 3.95%


### Set Expected repayment percentage per month

In [58]:
# Generate all month-end dates from June 2019 to December 2020
vintage_dates = pd.date_range(start='2019-06-30', end='2025-12-31', freq='ME')

In [59]:
# Create a new DataFrame indexed by future payment dates
all_payment_dates = pd.date_range(start='2019-07-31', end='2025-12-31', freq='ME')
forecast_matrix = pd.DataFrame(0.0, index=all_payment_dates.strftime('%d.%m.%Y'), columns=origination_df.index)

origination_df = pd.read_csv('Dataset.csv')
origination_df.rename(columns={'Unnamed: 0': 'Vintage'}, inplace=True)
origination_df.set_index('Vintage', inplace=True)
origination_df['Origination Amount'] = pd.to_numeric(origination_df['Origination Amount'])

# Apply expected repayments per vintage
expected_pct_values = expected_rep_percentages 

for vintage in origination_df.index:
    orig_amount = origination_df.loc[vintage, 'Origination Amount']
    for month_offset, pct in enumerate(expected_rep_percentages):
        try:
            pay_date = pd.to_datetime(vintage, dayfirst=True) + pd.DateOffset(months=month_offset+1)
            pay_str = pay_date.strftime('%d.%m.%Y')
            if pay_str in forecast_matrix.index:
                forecast_matrix.loc[pay_str, vintage] = orig_amount * (pct / 100)
        except:
            continue
# Sum all vintages to get total monthly forecast
forecasted_cash_flows_df = pd.DataFrame()
forecasted_cash_flows_df['Forecasted_Cash_Flow'] = forecast_matrix.sum(axis=1)


In [60]:
# Load and set up the origination data
origination_df = pd.read_csv('Dataset.csv')
origination_df.rename(columns={'Unnamed: 0': 'Vintage'}, inplace=True)
origination_df.set_index('Vintage', inplace=True)
origination_df['Origination Amount'] = pd.to_numeric(origination_df['Origination Amount'], errors='coerce')

# Expected repayment percentages over 20 months
expected_rep_percentages = [
    0.72, 2.30, 2.76, 3.05, 3.69, 3.73, 4.04, 4.25, 4.24, 4.21,
    4.32, 4.41, 4.33, 4.63, 4.55, 4.27, 4.45, 3.99, 4.21, 3.95
]

expected_pct_values = expected_rep_percentages

# Set up forecast matrix
all_payment_dates = pd.date_range(start='2019-07-31', end='2025-12-31', freq='ME')# frequency set to month end 
forecast_matrix = pd.DataFrame(0.0,index=all_payment_dates.strftime('%d.%m.%Y'), columns=origination_df.index)

# Apply expected repayment stream for each vintage
for vintage in origination_df.index:
    orig_amount = origination_df.loc[vintage, 'Origination Amount']
    vintage_date = pd.to_datetime(vintage, dayfirst=True)
    for month_offset, pct in enumerate(expected_pct_values):
        try:
            pay_date = vintage_date + pd.DateOffset(months=month_offset + 1)
            if (pay_date - vintage_date).days > 30 * 20:
                continue
            pay_str = pay_date.strftime('%d.%m.%Y')
            if pay_str in forecast_matrix.index:
                forecast_matrix.loc[pay_str, vintage] = orig_amount * (pct / 100)
        except:
            continue
        
forecasted_cash_flows_df = pd.DataFrame()
forecasted_cash_flows_df['Forecasted_Cash_Flow'] = forecast_matrix.sum(axis=1)
forecasted_cash_flows_df.index = pd.to_datetime(forecasted_cash_flows_df.index, format='%d.%m.%Y')
forecasted_cash_flows_df.sort_index(inplace=True)
forecasted_cash_flows_df = forecasted_cash_flows_df[
    forecasted_cash_flows_df.index > pd.to_datetime('2020-12-31')
]
            
# Sum cash flows across all vintages by month
forecasted_cash_flows_df = forecasted_cash_flows_df[
forecasted_cash_flows_df.index > pd.to_datetime('2020-12-31')
]

# Display results
print("Forecasted Cash Flows (Monthly Totals):")
print(formatted_cash_flows_df.head(10))


Forecasted Cash Flows (Monthly Totals):
           Forecasted_Cash_Flow
2021-01-31        $7,927,806.78
2021-02-28       $14,509,168.09
2021-03-31        $8,664,907.50
2021-04-30       $13,616,487.03
2021-05-31        $8,535,528.59
2021-06-30       $12,946,588.00
2021-07-31        $7,993,856.99
2021-08-31        $7,394,533.49
2021-09-30       $11,124,180.60
2021-10-31        $6,769,380.82


In [61]:
# Clean index and values for discounting
forecasted_cash_flows_df.index = pd.to_datetime(forecasted_cash_flows_df.index, format='%d.%m.%Y')
forecasted_cash_flows_df.sort_index(inplace=True)
# Set up discounting variables
annual_discount_rate = 0.05
monthly_discount_rate = (1 + annual_discount_rate) ** (1 / 12) - 1
valuation_date = pd.to_datetime('2020-12-31')

# Calculate month offsets from valuation date
months_diff = ((forecasted_cash_flows_df.index.year - valuation_date.year) * 12 +(forecasted_cash_flows_df.index.month - valuation_date.month))

# Calculate discount factors
discount_factors = (1 + monthly_discount_rate) ** (-months_diff)

# Calculate present value for each cash flow
present_values = forecasted_cash_flows_df['Forecasted_Cash_Flow'] * discount_factors

In [62]:
# Final present value of the portfolio
portfolio_value = round(present_values.sum(), 2)
print(f"Final Portfolio Valuation (Discounted to 31 Dec 2020): CHF {portfolio_value:,.2f}")

Final Portfolio Valuation (Discounted to 31 Dec 2020): CHF 137,876,240.63


The client’s estimate of the portfolio value was CHF 84’993’122.67. We will compute both the absolute and relative difference. The audit team considers any difference smaller than CHF 500’000 to be acceptable given the size of the portfolio.

In [63]:
# Client estimate
client_estimate = 84993122.67
absolute_diff = round(abs(portfolio_value - client_estimate), 2)
relative_diff = round((absolute_diff / client_estimate) * 100, 2)

print(f" Client Estimate: CHF {client_estimate:,.2f}")
print(f" Absolute Difference: CHF {absolute_diff:,.2f}")
print(f" Relative Difference: {relative_diff}%")

# Threshold check
threshold = 500000
if absolute_diff < threshold:
    print(" Within acceptable audit threshold (CHF 500,000).")
else:
    print(" Exceeds acceptable audit threshold.")

 Client Estimate: CHF 84,993,122.67
 Absolute Difference: CHF 52,883,117.96
 Relative Difference: 62.22%
 Exceeds acceptable audit threshold.


# Conclusion
Based on the expected repayment percentages provided and a 5% annual discount rate, our model estimates the portfolio value as of 31 December 2020 to be CHF 137,876,240.63, which exceeds the client’s estimate of CHF 84,993,122.67 by approximately CHF 52.88 million (+62.22%). 

The model suggests significantly higher value than the client’s estimate.

Forecasted repayments appear optimistic based on historical averages.

The client may be:

Using a more conservative repayment assumption

Accounting for defaults, charge-offs, or economic risk not modeled here

Our model assumes full repayment according to historical averages, whereas the client may have factored in expected credit losses, delinquency, or write-offs.

It is suggested that it might be helpful to reconcil assumptions with the client — particularly around repayment expectations, loss assumptions, and discounting methodology — to ensure alignment for audit validation and financial reporting.